# Week 9 Exercise: Interactive Visualization with Plotly

## Budget Execution Dataset

**Time:** 30 minutes

**Objective:** Create interactive visualizations using Plotly Express and add hover information and filters.

### What You Will Do:
1. Create a basic interactive scatter plot with Plotly
2. Add hover information to provide context on demand
3. Create an interactive bar chart with grouping
4. Add dropdown filters using Plotly's built-in filtering
5. Create a complete interactive visualization with 3 filter types

---

## Setup: Load Libraries and Data

In [ ]:
# Install plotly if needed (uncomment the line below)
# !pip install plotly

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("Libraries loaded successfully!")
print(f"Plotly version: {px.__version__}" if hasattr(px, '__version__') else "Plotly loaded")

In [ ]:
# Create sample Budget Execution dataset
# This simulates data from datos.gov.co - Ejecucion Presupuestal

np.random.seed(42)

departments = ['Education', 'Health', 'Infrastructure', 'Security', 'Social Services', 'Environment']
years = [2021, 2022, 2023, 2024]
categories = ['Personnel', 'Operations', 'Investment', 'Transfers']

data = []
for dept in departments:
    for year in years:
        for cat in categories:
            approved = np.random.randint(100000, 5000000)
            execution_rate = np.random.uniform(0.6, 1.0)
            executed = int(approved * execution_rate)
            
            data.append({
                'department': dept,
                'year': year,
                'category': cat,
                'budget_approved': approved,
                'budget_executed': executed,
                'execution_rate': round(execution_rate * 100, 1),
                'project_count': np.random.randint(5, 50)
            })

df = pd.DataFrame(data)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

In [ ]:
# Preview the data
df.head(10)

In [ ]:
# Basic statistics
df.describe()

---

## Task 1: Create a Basic Interactive Scatter Plot (5 minutes)

Create a scatter plot showing Approved Budget vs Executed Budget.

**Instructions:**
1. Use `px.scatter()` to create the plot
2. Set x-axis to approved budget and y-axis to executed budget
3. Color points by department
4. Add a meaningful title

In [ ]:
# Task 1: Basic scatter plot

# TODO: Create an interactive scatter plot
# Hint: px.scatter(df, x='column1', y='column2', color='category_column')

fig = px.scatter(
    df,
    x=___,  # approved budget column
    y=___,  # executed budget column
    color=___,  # department column
    title='Budget Execution: Approved vs Executed'
)

fig.show()

**Try the interactivity!**
- Hover over points to see values
- Click legend items to hide/show departments
- Use the toolbar to zoom, pan, and download

---

## Task 2: Add Rich Hover Information (5 minutes)

Enhance the scatter plot with detailed hover information.

**Instructions:**
1. Add `hover_name` to show the main identifier on hover
2. Add `hover_data` with additional columns
3. Add `size` parameter to encode a third variable

In [ ]:
# Task 2: Enhanced scatter plot with hover information

# TODO: Add hover information
# hover_name: main label (e.g., department)
# hover_data: list of additional columns to show
# size: column for bubble size

fig = px.scatter(
    df,
    x='budget_approved',
    y='budget_executed',
    color='department',
    size=___,  # Use execution_rate or project_count for bubble size
    hover_name=___,  # Main label shown on hover
    hover_data=[___],  # List of additional columns: ['year', 'category', 'execution_rate']
    title='Budget Execution with Hover Details'
)

# Update layout for better readability
fig.update_layout(
    xaxis_title='Approved Budget ($)',
    yaxis_title='Executed Budget ($)',
    legend_title='Department'
)

fig.show()

**Now hover over points and see:**
- The department name as the title
- Year, category, and execution rate in details
- Point size reflects execution rate

---

## Task 3: Create an Interactive Bar Chart (5 minutes)

Create a grouped bar chart showing budget by department and year.

**Instructions:**
1. Use `px.bar()` to create the chart
2. Group bars by year using the `color` parameter
3. Use `barmode='group'` for side-by-side bars

In [ ]:
# First, aggregate data by department and year
dept_year = df.groupby(['department', 'year']).agg({
    'budget_approved': 'sum',
    'budget_executed': 'sum',
    'project_count': 'sum'
}).reset_index()

dept_year['execution_rate'] = (dept_year['budget_executed'] / dept_year['budget_approved'] * 100).round(1)

dept_year.head(10)

In [ ]:
# Task 3: Grouped bar chart

# TODO: Create a grouped bar chart
# Hint: px.bar(df, x='category', y='value', color='group', barmode='group')

fig = px.bar(
    dept_year,
    x=___,  # department column
    y=___,  # budget_executed column
    color=___,  # year column (convert to string for categorical coloring)
    barmode=___,  # 'group' for side-by-side, 'stack' for stacked
    title='Budget Execution by Department and Year',
    hover_data=['budget_approved', 'execution_rate']
)

fig.update_layout(
    xaxis_title='Department',
    yaxis_title='Executed Budget ($)',
    legend_title='Year'
)

fig.show()

**Experiment with barmode:**
- Change `barmode='group'` to `barmode='stack'`
- Which is better for comparing departments vs comparing years?

---

## Task 4: Add Dropdown Filters with `animation_frame` (5 minutes)

Create a chart with year selector using Plotly's animation feature.

**Instructions:**
1. Use `animation_frame` parameter to add a year slider
2. The chart will animate through years

In [ ]:
# Task 4: Animated scatter plot with year slider

fig = px.scatter(
    df,
    x='budget_approved',
    y='budget_executed',
    color='department',
    size='project_count',
    hover_name='category',
    animation_frame=___,  # Add 'year' here to create year slider
    title='Budget Execution Over Time',
    range_x=[0, df['budget_approved'].max() * 1.1],
    range_y=[0, df['budget_executed'].max() * 1.1]
)

fig.update_layout(
    xaxis_title='Approved Budget ($)',
    yaxis_title='Executed Budget ($)'
)

fig.show()

**Use the slider:**
- Click Play to animate through years
- Drag the slider to specific years
- Notice how the legend filtering still works

---

## Task 5: Create Interactive Filters with Dropdown Buttons (10 minutes)

Create a chart with manual dropdown filters using `update_layout` and `updatemenus`.

This task shows you how to add custom filter controls.

In [ ]:
# Task 5: Custom dropdown filters

# We'll create 3 filter types:
# 1. Department filter (dropdown)
# 2. Category filter (buttons)
# 3. Year range (using facet)

# First, let's create a basic chart
fig = px.bar(
    df,
    x='department',
    y='budget_executed',
    color='category',
    barmode='stack',
    facet_col='year',  # Filter 3: Creates separate charts by year
    title='Budget Execution Dashboard',
    hover_data=['budget_approved', 'execution_rate']
)

fig.update_layout(
    height=500,
    xaxis_title='',
    yaxis_title='Executed Budget ($)'
)

# Rotate x-axis labels for readability
fig.update_xaxes(tickangle=45)

fig.show()

In [ ]:
# Advanced: Add dropdown buttons for category filter

# Create base figure with all categories
fig = go.Figure()

# Add traces for each category
categories = df['category'].unique()
colors = px.colors.qualitative.Plotly

for i, cat in enumerate(categories):
    cat_data = df[df['category'] == cat].groupby('department')['budget_executed'].sum().reset_index()
    fig.add_trace(
        go.Bar(
            name=cat,
            x=cat_data['department'],
            y=cat_data['budget_executed'],
            marker_color=colors[i % len(colors)]
        )
    )

# Create dropdown menu buttons
dropdown_buttons = [
    {'label': 'All Categories', 'method': 'update', 'args': [{'visible': [True] * len(categories)}]}
]

for i, cat in enumerate(categories):
    visibility = [False] * len(categories)
    visibility[i] = True
    dropdown_buttons.append(
        {'label': cat, 'method': 'update', 'args': [{'visible': visibility}]}
    )

# Add dropdown menu to layout
fig.update_layout(
    updatemenus=[
        {
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.1,
            'y': 1.15,
            'xanchor': 'left'
        }
    ],
    title='Budget by Department - Filter by Category',
    xaxis_title='Department',
    yaxis_title='Total Executed Budget ($)',
    barmode='stack',
    annotations=[
        {'text': 'Category:', 'x': 0, 'y': 1.12, 'xref': 'paper', 'yref': 'paper',
         'showarrow': False, 'font': {'size': 12}}
    ]
)

fig.show()

In [ ]:
# Final challenge: Create a chart with multiple filter types

# TODO: Complete this chart with 3 interactive elements
# 1. Use color to distinguish departments (click legend to filter)
# 2. Use facet_col to split by year
# 3. Use symbol to distinguish categories

fig = px.scatter(
    df,
    x='budget_approved',
    y='budget_executed',
    color=___,  # Filter 1: department (legend filtering)
    symbol=___,  # Filter 2: category (legend filtering)
    facet_col=___,  # Filter 3: year (separate panels)
    size='execution_rate',
    hover_data=['project_count'],
    title='Complete Budget Dashboard with 3 Filter Types'
)

fig.update_layout(height=500)
fig.show()

---

## Summary

### Key Plotly Skills Learned:

1. **`px.scatter()`** - Create interactive scatter plots
2. **`hover_name` and `hover_data`** - Add context on hover
3. **`px.bar()` with `barmode`** - Create grouped/stacked bar charts
4. **`animation_frame`** - Add time slider
5. **`facet_col`** - Split into multiple panels
6. **Legend clicking** - Filter by clicking legend items
7. **`updatemenus`** - Add custom dropdown filters

### Quick Reference:

```python
# Basic interactive scatter
fig = px.scatter(df, x='col1', y='col2', color='category')

# With hover information
fig = px.scatter(df, x='col1', y='col2', 
                 hover_name='name_col',
                 hover_data=['extra1', 'extra2'])

# Bar chart with grouping
fig = px.bar(df, x='cat', y='val', color='group', barmode='group')

# With time animation
fig = px.scatter(df, x='x', y='y', animation_frame='year')

# Display in Streamlit (next workshop!)
# st.plotly_chart(fig)
```

---

**Next:** In the workshop, you'll build a complete Streamlit dashboard with these charts!

*End of Exercise*